In [8]:
import numpy as np
import matplotlib.pyplot as plt
import PySimpleGUI as sg
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from scipy.signal import chirp, spectrogram
import scipy.fft as fft
import os
import scipy.io.wavfile as wavf

In [9]:
plt.style.use('Solarize_Light2')

# Helper Functions
#https://towardsdatascience.com/integrating-pyplot-and-pysimplegui-b68be606b960

# Produce a sine wave with a slider that determines its sampling rate

# VARS CONSTS:
_VARS = {'window': False,
         'fig_agg1': False,
         'fig_agg2': False,
         'RMSfig_agg': False,
         'PowerSpecfig_agg': False,
         'TimeSeriesfig_agg': False,
         'pltFig1': False,
         'pltFig2': False,
         'RMSFig': False,
         'PowerSpecFig': False,
         'TimeSeriesFig': False,
         'DAQ_Cntd_colour': 'gray',
         'sensitivity': 1,
         'noise_power_bandwidth': 1,
         'directory_name': False,
         'in_use_directory_name': False,
         'sampling_rate': 44100,
         'max_frequency': 800,
         'frequency': 800,
         'peak_amplitude': 1,
         'pulse_time': 1,
         'pulse_type': 'Constant',
         'duration' : 1,
         'freq_resolution': 0,
         'recorded_data': 0,
        }

pulse_choices = ('Constant', 'Chirp')

plt.style.use('Solarize_Light2')


# Helper Functions

def generate_directory_name(name, x=0):
    _VARS['directory_name'] = name
    while True:
        dir_name = (_VARS['directory_name'] + (' ' + str(x) if x != 0 else '')).strip()
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
            _VARS['in_use_directory_name'] = dir_name
            return dir_name
        else:
            x = x + 1
    #os.chdir(_VARS['directory_name']) 

    
#def query_devices():
#    local = ni.system.System.local()
#    for device in local.devices:
#        connected_ai = [chan.name for chan in device.ai_physical_chans]


def updateDAQConnected():
    _VARS['DAQ_Cntd_colour'] = 'Green'

def draw_figure(canvas, figure):
    figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack(side='top', fill='both', expand=1)
    return figure_canvas_agg


# \\  -------- PYSIMPLEGUI -------- //

AppFont = 'Any 16'
SliderFont = 'Any 14'
sg.theme('black')



# \\  -------- PYPLOT -------- //


def makeSynthData():
    if _VARS['pulse_type'] == 'Constant':      
        sampling_interval = 1/_VARS['sampling_rate']

        freq = 3
        omega = 2*np.pi*_VARS['frequency']
        amplitude = np.iinfo(np.int16).max # output machine limit for type

        xData = np.arange(0, _VARS['pulse_time'], sampling_interval)
        yData = _VARS['peak_amplitude']*np.sin(omega*xData)
        
        

    if _VARS['pulse_type'] == 'Chirp':
        sampling_interval = 1/_VARS['sampling_rate']
        
        amplitude = np.iinfo(np.int16).max # output machine limit for type
        
        xData = np.arange(0, _VARS['pulse_time'], sampling_interval) # 4-second chirp    
        yData = _VARS['peak_amplitude']*chirp(xData, f0=100, f1=_VARS['max_frequency'], \
                    t1=_VARS['pulse_time'], method='linear') #  f0 = initial freq, f1 = freq at t1
        
    FFT = fft.rfft(yData)
    psdData = 10*np.log(np.abs(FFT)**2) - 10*np.log(1e-6) + 20*np.log(_VARS['sensitivity']) \
    - 10*np.log(_VARS['noise_power_bandwidth']) 
    freqData = fft.rfftfreq(xData.size, 1/_VARS['sampling_rate']) # (window length, sample spacing)
        
    return (xData, yData , freqData, psdData)


def recordADC():

    # Dummy data before working with DAQ
    
    #sampling_interval = 1/_VARS['sampling_rate']
    #freq = _VARS['frequency']
    #omega = 2*np.pi*freq
    #amplitude = 1
    #amplitude = np.iinfo(np.int16).max # output machine limit for type
    #xData = np.arange(0, _VARS['pulse_time'], sampling_interval)
    #yData = _VARS['peak_amplitude']*amplitude*np.sin(omega*xData)
    dataXY = makeSynthData()
    yData = dataXY[1]
    xData = dataXY[0]
    rec_yData = playrec(yData, _VARS['sampling_rate'])
    #print(rec_yData)
    data = np.column_stack((xData,rec_yData))
    _VARS['recorded_data'] = data
    
    global i, k
    os.chdir(os.path.join(path, _VARS['in_use_directory_name']))  
    
    while os.path.exists("recording%s.npy" % i):
        i += 1
        print("i = ", i)
    with open(os.path.join(path, _VARS['in_use_directory_name'], "recording%s.npy" % i), 'wb') as f:
        np.save(f, data)
                 
    while os.path.exists("recording%s.wav" % k):
        k += 1
        print("k = ", k)
    Rx = os.path.join(path, _VARS['in_use_directory_name'], "recording%s.wav" % k)
    wavf.write(Rx, int(_VARS['sampling_rate']), _VARS['recorded_data'])
    

    global j    
        
    if j == 0:
        drawTimeSeriesRx()
        drawPSDRx()
        j += 1
    else:
        updateMeasuredChart()
        
# https://github.com/ni/nidaqmx-python/commit/3e1261a99338d5c1f9dd3f695d2f68493f756b0e

def playrec(data, samplerate):

    data = np.asarray(data).T
    nsamples = data.shape[0]

    with ni.Task() as read_task, ni.Task() as write_task:

        aochan = write_task.ao_channels.add_ao_voltage_chan("cDAQ1Mod2/ao0",'output', -2, 2)

        read_task.ai_channels.add_ai_voltage_chan("cDAQ1Mod1/ai0")

        for task in (read_task, write_task):
            task.timing.cfg_samp_clk_timing(rate=samplerate, source='OnboardClock',
                                        sample_mode=ni.constants.AcquisitionType.FINITE, 
                                      samps_per_chan=nsamples)
    
            # commiting task reduces lag between read and write
        read_task.control(TaskMode.TASK_COMMIT)
        write_task.control(TaskMode.TASK_COMMIT)
    
        write_task.triggers.start_trigger.cfg_dig_edge_start_trig(read_task.triggers.start_trigger.term)
        write_task.write(data, auto_start=False)
        write_task.start()
        indata = np.asarray(read_task.read(nsamples)).T
    


    return np.asarray(indata).T

def drawTimeSeriesRx():
    _VARS['TimeSeriesFig'] = plt.figure(figsize=(12.8, 4.8))
    x = _VARS['recorded_data'][:, 0]
    y = _VARS['recorded_data'][:, 1]
    plt.plot(x,y)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (V)')
    plt.title('Signal - time series')
    _VARS['TimeSeriesfig_agg'] = draw_figure(
        _VARS['window']['TimeSeriesCanvas'].TKCanvas, _VARS['TimeSeriesFig'])
     

def drawPSDRx():
    _VARS['PowerSpecFig'] = plt.figure()
    x = _VARS['recorded_data'][:, 0]
    y = _VARS['recorded_data'][:, 1]
    FFT = fft.rfft(y)
    psdData = 10*np.log(np.abs(FFT)**2) - 10*np.log(1e-6) + 20*np.log(_VARS['sensitivity']) \
    - 10*np.log(_VARS['noise_power_bandwidth'])
    # Power spectral density(above) we need to include sensitivity on first tab
    # and consider the noise power bandwidth
    freqData = fft.rfftfreq(x.size, 1/_VARS['sampling_rate']) # (window length, sample spacing)
    idx = np.argsort(freqData)
    plt.plot(freqData[idx], psdData[idx]) #, '.k')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD (dB re 1uPa2/Hz)')
    plt.title('Signal - power spectral density')
    _VARS['PowerSpecfig_agg'] = draw_figure(
        _VARS['window']['PowerSpecCanvas'].TKCanvas, _VARS['PowerSpecFig'])

def drawRMS():
    _VARS['RMSFig'] = plt.figure()
    x = np.linspace(0,1,5)
    y = 2*x
    plt.plot(x,y)
    _VARS['RMSfig_agg'] = draw_figure(
        _VARS['window']['RMSCanvas'].TKCanvas, _VARS['RMSFig'])
          
def drawTimeSeriesTx():

    _VARS['pltFig1'] = plt.figure()
    dataXY = makeSynthData()

    plt.plot(dataXY[0], dataXY[1]) #, '.k')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (V)')
    plt.title('Signal - time series')
    _VARS['fig_agg1'] = draw_figure(
        _VARS['window']['figCanvas'].TKCanvas, _VARS['pltFig1'])
    
def drawPSDTx():
    _VARS['pltFig2'] = plt.figure()
    
    dataXY = makeSynthData()
    idx = np.argsort(dataXY[2])
    freqs = dataXY[2]
    PSD = dataXY[3]

    plt.plot(freqs[idx], PSD[idx]) #, '.k')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD (dB re 1uPa2/Hz)')
    plt.title('Signal - power spectral density')
    _VARS['fig_agg2'] = draw_figure(
        _VARS['window']['figCanvasFreq'].TKCanvas, _VARS['pltFig2'])
       
def updateMeasuredChart():
    _VARS['TimeSeriesFig'] = plt.figure(figsize=(12.8, 4.8))
    _VARS['TimeSeriesfig_agg'].get_tk_widget().forget()
    x = _VARS['recorded_data'][:, 0]
    y = _VARS['recorded_data'][:, 1]
    # plt.cla()
    plt.clf()
    plt.plot(x,y)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (V)')
    plt.title('Signal - time series')
    _VARS['TimeSeriesfig_agg'] = draw_figure(
        _VARS['window']['TimeSeriesCanvas'].TKCanvas, _VARS['TimeSeriesFig'])
    
    _VARS['PowerSpecFig'] = plt.figure()
    _VARS['PowerSpecfig_agg'].get_tk_widget().forget()    
    plt.clf()
    FFT = fft.rfft(y)
    psdData = 10*np.log(np.abs(FFT)**2) # Power spectral density
    freqData = fft.rfftfreq(x.size, 1/_VARS['sampling_rate']) # (window length, sample spacing)
    idx = np.argsort(freqData)
    
    plt.plot(freqData[idx], psdData[idx]) #, '.k')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD (dB re 1uPa2/Hz)')
    plt.title('Signal - power spectral density')
    _VARS['PowerSpecfig_agg'] = draw_figure(
        _VARS['window']['PowerSpecCanvas'].TKCanvas, _VARS['PowerSpecFig'])

def updateChart():
    _VARS['pltFig1'] = plt.figure()
    _VARS['fig_agg1'].get_tk_widget().forget()
    
    dataXY = makeSynthData()
    # plt.cla()
    plt.clf()
    plt.plot(dataXY[0], dataXY[1]) #, '.k')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (V)')
    plt.title('Signal - time series')
    _VARS['fig_agg1'] = draw_figure(
        _VARS['window']['figCanvas'].TKCanvas, _VARS['pltFig1'])
    
    _VARS['pltFig2'] = plt.figure()
    _VARS['fig_agg2'].get_tk_widget().forget()    
    plt.clf()
    idx = np.argsort(dataXY[2])
    freqs = dataXY[2]
    PSD = dataXY[3]
    plt.plot(freqs[idx], PSD[idx]) #, '.k')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD (dB re 1uPa2/Hz)')
    plt.title('Signal - power spectral density')
    _VARS['fig_agg2'] = draw_figure(
        _VARS['window']['figCanvasFreq'].TKCanvas, _VARS['pltFig2'])

def updateData(val):
    _VARS['sampling_rate'] = val
    updateChart()
    
def updateParameters(max_freq, freq, peak_amp, s_rate, p_time, p_type):

    _VARS['max_frequency'] = max_freq
    _VARS['frequency'] = freq
    _VARS['peak_amplitude'] = peak_amp
    _VARS['sampling_rate'] = s_rate
    _VARS['pulse_time'] = p_time
    p_type = p_type.replace("['" ,"")
    p_type = p_type.replace("']" ,"")
    if p_type == " ":
        _VARS['pulse_type'] = _VARS['pulse_type']
    else:
        _VARS['pulse_type'] = p_type  
    updateChart()

# \\  -------- PYPLOT -------- //



In [10]:
path = os.getcwd()
print ("The current working directory is %s" % path)

The current working directory is /Users/iandownie/Hydrophones/Acoustic Analysis
